
---

📚 <font color='blue'> Based on tutorial
[Object Detection with TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/tutorials/model_maker_object_detection)</font>

---



---

### Download model from Detection Model Zoo

Look at [Mobile models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md#mobile-models) section, model name is `ssd_mobilenet_v3_small_coco`.

By default, it will be downloaded to `/content/` folder.

---


In [4]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz

--2021-09-07 09:51:34--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.70.128, 2607:f8b0:4001:c02::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.70.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27341125 (26M) [application/gzip]
Saving to: ‘ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz’

ssd_mobilenet_v3_sm 100%[===================>]  26.07M   137MB/s    in 0.2s    

2021-09-07 09:51:34 (137 MB/s) - ‘ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz’ saved [27341125/27341125]




---

### Untar tar.gz file

Once it is done, you can see `*.tflite` model available at `/content/ssd_mobilenet_v3_small_coco_2020_01_14/model.tflite`

---


In [5]:
!tar -xzvf "/content/ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz"

ssd_mobilenet_v3_small_coco_2020_01_14/
ssd_mobilenet_v3_small_coco_2020_01_14/model.ckpt.index
ssd_mobilenet_v3_small_coco_2020_01_14/frozen_inference_graph.pb
ssd_mobilenet_v3_small_coco_2020_01_14/model.ckpt.data-00000-of-00001
ssd_mobilenet_v3_small_coco_2020_01_14/model.ckpt.meta
ssd_mobilenet_v3_small_coco_2020_01_14/pipeline.config
ssd_mobilenet_v3_small_coco_2020_01_14/checkpoint
ssd_mobilenet_v3_small_coco_2020_01_14/model.tflite



---

### Import required packages

---


In [6]:
import numpy as np
import os

import tensorflow as tf
assert tf.__version__.startswith('2')
print("Current TF version:", tf.__version__)

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

Current TF version: 2.6.0



---

### Load the trained TFLite model and define some visualization functions

---


In [7]:
# Load the trained TFLite model and define some visualization functions

import cv2

from PIL import Image

model_path = '/content/ssd_mobilenet_v3_small_coco_2020_01_14/model.tflite'

# Load the labels into a list
# https://github.com/amikelive/coco-labels/blob/master/coco-labels-paper.txt
classes = ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train",
           "truck", "boat", "traffic light", "fire hydrant", "???(ID12)", "stop sign",
           "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow",
           "elephant", "bear", "zebra", "giraffe", "???(ID26)", "backpack", "umbrella",
           "???(ID29)", "???(ID30)", "handbag", "tie", "suitcase", "frisbee", "skis",
           "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
           "skateboard", "surfboard", "tennis racket", "bottle", "???(ID45)", "wine glass",
           "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich",
           "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
           "chair", "couch", "potted plant", "bed", "???(ID66)", "dining table",
           "???(ID68)", "???(ID69)", "toilet", "???(ID71)", "tv", "laptop", "mouse", "remote",
           "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
           "refrigerator", "???(ID83)", "book", "clock", "vase", "scissors",
           "teddy bear", "hair drier", "toothbrush"]

# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  return resized_img, original_image


def set_input_tensor(interpreter, image):
  """Set the input tensor."""
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image


def get_output_tensor(interpreter, index):
  """Retur the output tensor at the given index."""
  output_details = interpreter.get_output_details()[index]
  tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
  return tensor


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""
  # Feed the input image to the model
  set_input_tensor(interpreter, image)
  interpreter.invoke()

  # Get all outputs from the model
  boxes = get_output_tensor(interpreter, 0)
  classes = get_output_tensor(interpreter, 1)
  scores = get_output_tensor(interpreter, 2)
  count = int(get_output_tensor(interpreter, 3))

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8


---

### Run object detection and show the detection results

---


In [8]:
# Run object detection and show the detection results

# INPUT_IMAGE_URL = "https://c8.alamy.com/comp/2BYP7FK/little-girl-hugging-teddy-bear-2BYP7FK.jpg" #@param {type:"string"}
# INPUT_IMAGE_URL = "https://vw.plichta.com.pl/wp-content/uploads/2017/07/db2016au00964.jpg" #@param {type:"string"}
# INPUT_IMAGE_URL = "https://www.indianhealthyrecipes.com/wp-content/uploads/2018/09/veg-mayo-sandwich-recipe.jpg" #@param {type:"string"}
# INPUT_IMAGE_URL = "https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/apples-vs-oranges-health-benefits-1524606349.jpg" #@param {type:"string"}
INPUT_IMAGE_URL = "https://messalonskeehs.files.wordpress.com/2013/02/screen-shot-2013-02-06-at-10-50-37-pm.png" #@param {type:"string"}
# INPUT_IMAGE_URL = "https://storage.googleapis.com/cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg" #@param {type:"string"}
DETECTION_THRESHOLD =  0.45#@param {type:"number"}

TEMP_FILE = '/tmp/image.png' 

!wget -q -O $TEMP_FILE $INPUT_IMAGE_URL
im = Image.open(TEMP_FILE)
im.thumbnail((512, 512), Image.ANTIALIAS)
im.save(TEMP_FILE, 'PNG')

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    TEMP_FILE,
    interpreter,
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
Image.fromarray(detection_result_image)


---

### Read model signatures

Explanation for model input/output available [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/running_on_mobile_tf2.md)

```
Four Outputs:
  detection_boxes: a float32 tensor of shape [1, num_boxes, 4] with box locations
  detection_classes: a float32 tensor of shape [1, num_boxes] with class indices
  detection_scores: a float32 tensor of shape [1, num_boxes] with class scores
  num_boxes: a float32 tensor of size 1 containing the number of detected boxes
```

---


In [9]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
print("INPUT DETAILS", input_details)
output_details = interpreter.get_output_details()
print("OUTPUT DETAILS", output_details)

INPUT DETAILS [{'name': 'normalized_input_image_tensor', 'index': 383, 'shape': array([  1, 320, 320,   3], dtype=int32), 'shape_signature': array([  1, 320, 320,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.0078125, 128), 'quantization_parameters': {'scales': array([0.0078125], dtype=float32), 'zero_points': array([128], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
OUTPUT DETAILS [{'name': 'TFLite_Detection_PostProcess', 'index': 375, 'shape': array([  1, 100,   4], dtype=int32), 'shape_signature': array([  1, 100,   4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'TFLite_Detection_PostProcess:1', 'index': 376, 'shape': array([  1, 100], dtype=int32), 'shape_signature': array([  1, 100], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quanti

In [10]:
output_shape_box_locations = output_details[0]['shape']
print("OUTPUT SHAPE - BOX LOCATIONS    ", output_shape_box_locations)
output_shape_class_indices = output_details[1]['shape']
print("OUTPUT SHAPE - CLASS INDICES    ", output_shape_class_indices)
output_shape_class_scores = output_details[2]['shape']
print("OUTPUT SHAPE - CLASS SCORES     ", output_shape_class_scores)
output_shape_detected_boxes_no = output_details[3]['shape']
print("OUTPUT SHAPE - DETECTED BOXES NO", output_shape_detected_boxes_no)

OUTPUT SHAPE - BOX LOCATIONS     [  1 100   4]
OUTPUT SHAPE - CLASS INDICES     [  1 100]
OUTPUT SHAPE - CLASS SCORES      [  1 100]
OUTPUT SHAPE - DETECTED BOXES NO [1]



---

### Get all outputs from the model

---


In [11]:
# Box locations
boxes = get_output_tensor(interpreter, 0)
print(boxes)
print(boxes.shape, boxes.dtype)

[[ 4.33939695e-03  1.61061212e-01  6.04829490e-01  5.65723419e-01]
 [ 6.66867793e-02  5.56703866e-01  6.14518046e-01  9.06734765e-01]
 [ 4.39195514e-01  4.48986888e-03  9.76666927e-01  9.78163481e-01]
 [ 3.92198563e-02  9.11876559e-03  1.00149703e+00  9.67847824e-01]
 [ 2.97930241e-02  4.97591734e-01  7.84798205e-01  9.72207189e-01]
 [ 6.76889002e-01  3.58340144e-03  9.95600641e-01  9.76433039e-01]
 [ 5.60851693e-02  6.86522424e-01  6.25434041e-01  9.22787964e-01]
 [ 2.87892431e-01  1.49135932e-01  5.99198461e-01  5.54012239e-01]
 [ 1.21212602e-02  9.67231095e-02  6.19290173e-01  9.38111901e-01]
 [ 8.12320411e-02  5.77063084e-01  3.70171785e-01  7.55319834e-01]
 [ 3.51412058e-01  1.52800679e-02  6.14188254e-01  1.46247283e-01]
 [ 1.57340765e-02  1.69683635e-01  3.61433417e-01  5.64319611e-01]
 [ 3.10986608e-01  5.60995936e-01  6.33912325e-01  9.15501714e-01]
 [ 1.55757248e-01  8.39949846e-02  7.21754134e-01  5.67400455e-01]
 [ 1.17549062e-01  5.55261314e-01  4.48880970e-01  7.26126850e

In [12]:
# Class indices
classes = get_output_tensor(interpreter, 1)
print(classes)
print(classes.shape, classes.dtype)

[54. 52. 51. 66. 52. 50. 52. 54. 52. 52. 52. 54. 52. 54. 52. 54. 52. 52.
 54. 52. 54. 51. 54. 52. 54. 52. 52. 51. 52. 54. 51. 54. 54. 52. 52. 52.
 51. 52. 52. 51. 52. 51. 54. 52. 52. 54. 54. 54. 52. 54. 54. 51. 54. 52.
 50. 54. 54. 52. 52. 54. 52. 52. 52. 52. 52. 52. 52. 52. 56. 66. 52. 54.
 54. 51. 66. 52. 52. 54. 54. 54. 52. 52. 52. 51. 51. 54. 52. 54. 52. 52.
 52. 49. 54. 52. 54. 51. 52. 52. 52. 54.]
(100,) float32


In [13]:
# Class scores
scores = get_output_tensor(interpreter, 2)
print(scores)
print(scores.shape, scores.dtype)

[0.75766706 0.7211507  0.55102885 0.43742114 0.23232132 0.21212402
 0.1918014  0.17807108 0.17705    0.1714167  0.16949806 0.16868451
 0.1676094  0.16311604 0.15703487 0.15388957 0.15294671 0.15276551
 0.14605448 0.13956171 0.13895217 0.13821998 0.13613528 0.13171759
 0.13005129 0.13001019 0.12752762 0.1259543  0.12126285 0.11774331
 0.1145561  0.11186761 0.1101841  0.10739997 0.10708117 0.10611433
 0.10568178 0.10539427 0.1043914  0.10197011 0.10073888 0.10071567
 0.09784496 0.09592673 0.09563312 0.09556741 0.09537482 0.09417838
 0.09416649 0.0926207  0.09151053 0.09136385 0.09101349 0.09050223
 0.09042397 0.08971849 0.08862516 0.08861956 0.0874638  0.08510062
 0.08496532 0.08472702 0.08252987 0.08182544 0.08098543 0.07997772
 0.07929879 0.07910466 0.07907212 0.07906872 0.0787043  0.07850495
 0.07793197 0.07659915 0.07611364 0.07591352 0.07588923 0.0750151
 0.0748316  0.07430965 0.07409659 0.07408378 0.07377297 0.07327002
 0.07321182 0.07276416 0.07240522 0.07213113 0.07204655 0.07099

In [14]:
# Detected boxes no
count = int(get_output_tensor(interpreter, 3))
print(count)

100



---

### Final comments

Assuming
```
INPUT_IMAGE_URL = "https://messalonskeehs.files.wordpress.com/2013/02/screen-shot-2013-02-06-at-10-50-37-pm.png" #@param {type:"string"}
```
you get first three `classes` such as:<br>
`[54. 52. 51.]`,<br>
and corresponding `scores` such as:<br>
`[0.75766706 0.7211507  0.55102885]`.

Based on that and COCO labels, the results are:<br>
`54 = orange`, `52 = apple`, `51 = banana`.<br>
And confidence levels:<br>
`76% for orange`, `72% for apple`, `55% for banana` which you can see also in image. 

---
